In [ ]:
# !python -m pip install -U tifffile[all]
# !pip list
# Install dependencies
!python -m pip install --upgrade pip
!pip install tensorflow

In [1]:
import numpy as np
import tensorflow as tf 
from tensorflow.keras import layers, Model
from osgeo import gdal
import json
import matplotlib.pyplot as plt
import tifffile as tiff
import os
import pandas as pd
from skimage.transform import resize
from sklearn.model_selection import train_test_split
import re
from datetime import datetime

2024-04-28 19:14:46.419370: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-28 19:14:46.421567: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-28 19:14:46.466774: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-28 19:14:46.467597: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-28 19:14:47.179031: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not fin

In [2]:
def load_json_data(json_file):
    with open(json_file, 'r') as file: 
        data = json.load(file)
    return data 

data_s1 = load_json_data('/home/pedro/Documents/JADS/DeepLearning/SEN12FLOOD/S1list.json')
data_s2 = load_json_data('/home/pedro/Documents/JADS/DeepLearning/SEN12FLOOD/S2list.json')

In [3]:
# ## load images as we saw on kaggle
# img = tiff.imread('/home/pedro/Documents/JADS/DeepLearning/SEN12FLOOD/0066/S2_2019-02-04_B01.tif')
# img_array = np.array(img)
# plt.imshow(img)
# print(img_array.shape)            

In [4]:
### This cell is for read the tiff images using the tiff python library
### Images, filenames and flooding labels are set to be stored in arrays while reading the images
### Function takes a particular directory (A Flloding scenario and fetch for each label in the flooding json files)
### It automatically fetch for the number of the folder being processed 
### And retrieve both the name of the file and the flooding label. e.g from the sequence:
### 
###
### {"0063": {"1": {"date": "2019-02-04", "FLOODING": false, "FULL-DATA-COVERAGE": true, "filename": "S2_2019-02-04"}, "count": 1, "folder": "0063", "geo": {"type": "Polygon", "coordinates": [[[28.29722, -15.382762], [28.297507, -15.429039], [28.345216, -15.428755], [28.344918, -15.382479], [28.29722, -15.382762]]]}},
### Based on the name of the folder "0063" it reads the images of the folder and it appends both filename and flooding label.
### To cross and validate whether there is flooding or not the folder "0200" is used on the next cell. 
### This is the second sequence in the S2 json data. Easy to validate

In [5]:
def load_images_from_directory(directory, s1_data, s2_data):
    images = []
    filenames = []
    labels = []
    
    folder_name = os.path.basename(directory)
    print(f"Processing folder: {folder_name}")

    for file_name in os.listdir(directory):
        if file_name.endswith('.tif') or file_name.endswith('.tiff'):
            json_data = s1_data if file_name.startswith('S1') else s2_data if file_name.startswith('S2') else None
            if json_data is None: 
                print(f"Skipping file (not S1 or S2): {file_name}")
                continue
            
            found = False
            for item in json_data.get(folder_name, {}).values():
                if isinstance(item, dict) and 'filename' in item and item['filename'] in file_name:
                    file_path = os.path.join(directory, file_name)
                    img = tiff.imread(file_path)
                    img_array = np.array(img)
                    images.append(img_array)
                    filenames.append(file_name)
                    labels.append(item.get('FLOODING', False))
                    found = True 
                    break
    return images, filenames, labels

In [6]:
### Definition to extract the date and order by date so we respect time-series nature for the model

In [7]:
def extract_date(filename):
    # Try to extract S2 date format first
    match = re.search(r'\d{4}-\d{2}-\d{2}', filename)
    if match:
        return datetime.strptime(match.group(), '%Y-%m-%d').date()
    else:
        # Try the S1 date format
        match = re.search(r'\d{8}T\d{6}', filename)
        if match:
            return datetime.strptime(match.group(), '%Y%m%dT%H%M%S').date()
    return None  # Important to handle cases where no date is found

In [8]:
### Following code is to get a list of all the flooding events (folders)
### present on the daataset. There is also code to select which folders are
### going to be selected to train and test the model

In [9]:
#### This code is to apply recursively the function to read the images to all 
### The folders present on the dataset. As the number of the sequence event is stored in a 
### dictionary everythins is stored on the same dictionary with sequence and flooding label

def process_folders(folder_list, main_directory, s1_data, s2_data):
    scenario_data = {}
    
    for folder in folder_list:
        directory = os.path.join(main_directory, folder)
        images, filenames, labels = load_images_from_directory(directory, s1_data, s2_data)

        if not images:
            continue
        
        # Collect data and sort it by date
        temp_data = [(img, fname, lbl, extract_date(fname)) for img, fname, lbl in zip(images, filenames, labels) if extract_date(fname) is not None]
        temp_data.sort(key=lambda x: x[3]) #Sort by date
        
        scenario_data[folder] = {
            'images': [data[0] for data in temp_data],
            'filenames': [data[1] for data in temp_data],
            'labels': [data[2] for data in temp_data],
            'dates': [data[3] for data in temp_data]
        }
            
    return scenario_data

In [10]:
def list_folders(main_directory):
    return [f for f in os.listdir(main_directory) if os.path.isdir(os.path.join(main_directory, f))]

# Main SEN12FLOOD directory, whole data is there
main_directory = '/home/pedro/Documents/JADS/DeepLearning/SEN12FLOOD'

# List all folders
all_folders = list_folders(main_directory)

# Manually select folders for training and testing
# test_folders = [str(i) for i in range(68)]  # Example folders for training
# train_folders = ['0001', '0002', '0003', '0004', '0005', '0006', '0007', '0008', '0009', '0010', '0011', '0012', '0013', '0014', '0015', '0016', '0018', '0020', '0021', '0022', '0023', '0024', '0025', '0026', '0027', '0028', '0029', '0030', '0031', '0033', '0034', '0035', '0036', '0037', '0042', '0043', '0044', '0045', '0046', '0047', '0048', '0050', '0053', '0054', '0055', '0057', '0059', '0060', '0061', '0063', '0065', '0066', '0067', '0068', '0069', '0070', '0071', '0072', '0073', '0074', '0075', '0076', '0077', '0079', '0080', '0081', '0082', '0084', '0085', '0086', '0088', '0089', '0090', '0091', '0093', '0094', '0095', '0096', '0097', '0098', '0099', '0100', '0101', '0102', '0103', '0104', '0105', '0106', '0107', '0108', '0109', '0111', '0115', '0116', '0117', '0118', '0120', '0121', '0122', '0123', '0124', '0125', '0126', '0127', '0128', '0130', '0131', '0132', '0133', '0134', '0135', '0137', '0138', '0139', '0140', '0141', '0143', '0144', '0145', '0146', '0147', '0148', '0149', '0150', '0151', '0154', '0155', '0156', '0157', '0158', '0159', '0160', '0161', '0162', '0163', '0165', '0166', '0167', '0168', '0169', '0170', '0171', '0173', '0174', '0176', '0177', '0178', '0181', '0182', '0184', '0186', '0187', '0188', '0191', '0192', '0193', '0194', '0196', '0198', '0199', '0200', '0201', '0203', '0204', '0205', '0206', '0207', '0208', '0209', '0210', '0212', '0213', '0214', '0215', '0216', '0217', '0218', '0219', '0220', '0221', '0222', '0223', '0225', '0226', '0227', ' 0229', '0230', '0231', '0232', '0233', '0234', '0235', '0236', '0238', '0240', '0241', '0243', '0244', '0245', '0246', '0247', '0248', '0249', '0250', '0253', '0254', '0255', '0256', '0257', '0258', '0259', '0260', '0261', '0262', '0263', '0266', '0267', '0271', '0272', '0273', '0274', '0275', '0276', '0277', '0278', '0279', '0280', '0281', '0282', '0285', '0286', '0287', '0288', '0290', '0293', '0294', '0295', '0296', '0298', '0299', '0300', '0301', '0303', ' 0304', '0305', '0306', '0307', '0308', '0309', '0310', '0311', '0313', '0316', '0318', '0319', '0320', '0321', '0323', '0324', '0325', '0326', '0327', '0328', '0329', '0330', '0331', '0332', '0333', '0334', '0335', '0336']   # Example folders for testing
train_folders = ['0018']
test_folders = ['24']
validation_folders = ['30']

In [11]:
# KEEPNG THE CODE BELOW. DISCUSS IT WITH PEDRO

In [12]:
# ## Preprocess train and test folders
# train_images, train_labels, train_scenario_labels = process_folders(train_folders, main_directory, data_s1, data_s2)
# test_images, test_labels, test_scenario_labels = process_folders(test_folders, main_directory, data_s1, data_s2)
# validation_images, validation_labels, validation_scenario_labels = process_folders(validation_folders, main_directory, data_s1, data_s2)

In [13]:
### Preprocess train and test folders
train_data = process_folders(train_folders, main_directory, data_s1, data_s2)
test_data = process_folders(test_folders, main_directory, data_s1, data_s2)
validation_data = process_folders(validation_folders, main_directory, data_s1, data_s2)

# Example of accessing data

Processing folder: 0018
Processing folder: 24
Processing folder: 30


In [14]:
### NP.array and resize images

In [15]:
from skimage.transform import resize

def resize_and_convert_images(grouped_data, target_shape):
    for scenario, data in grouped_data.items():
        # Resize images and explicitly convert to NumPy arrays if not already
        data['images'] = [np.array(resize(img, target_shape, preserve_range=True), dtype=np.float32) for img in data['images']]
    return grouped_data

In [16]:
target_shape = (522, 544)  # Define the target shape for resizing images
train_data_resized = resize_and_convert_images(train_data, target_shape)
test_data_resized = resize_and_convert_images(test_data, target_shape)
validation_data_resized = resize_and_convert_images(validation_data, target_shape)

In [17]:
#### Structure the data to prepare the input for the ConvLSTM neural network

In [18]:
import numpy as np 
from collections import defaultdict 

def group_by_scenario_and_type(scenario_data):
    grouped_data = defaultdict(lambda: {'S1': [], 'S2': [], 'labels': []})
    for scenario, data in scenario_data.items():
        for img, lbl, fname in zip(data['images'], data['labels'], data['filenames']):
            if 'S1' in fname:
                grouped_data[scenario]['S1'].append(img)
            elif 'S2' in fname:
                grouped_data[scenario]['S2'].append(img)
            grouped_data[scenario]['labels'].append(lbl)  # Assuming labels are tied to images
    return grouped_data

In [19]:
train_grouped = group_by_scenario_and_type(train_data_resized)
test_grouped = group_by_scenario_and_type(test_data_resized)
validation_grouped = group_by_scenario_and_type(validation_data_resized)

In [20]:
def prepare_data_for_model(scenario_data, target_shape):
    X_s1, X_s2, y = [], [], []

    for scenario, data in scenario_data.items():
        # Since images might already be resized, check if resizing is needed again
        images_s1 = [resize(img, target_shape, preserve_range=True, anti_aliasing=True) if img.shape[:2] != target_shape else img for img in data['S1']]
        images_s2 = [resize(img, target_shape, preserve_range=True, anti_aliasing=True) if img.shape[:2] != target_shape else img for img in data['S2']]
        labels = data['labels']

        X_s1.append(np.array(images_s1))
        X_s2.append(np.array(images_s2))
        y.append(np.array(labels))

    # Concatenate arrays from each scenario to form a continuous array
    X_s1 = np.concatenate(X_s1, axis=0) if X_s1 else np.array([], dtype=np.float32)
    X_s2 = np.concatenate(X_s2, axis=0) if X_s2 else np.array([], dtype=np.float32)
    y = np.concatenate(y, axis=0) if y else np.array([], dtype=np.float32)

    return X_s1, X_s2, y

In [21]:
#### Erase the previous model. Strategy must be to train, test and validate model 
#### By sequences and not by all te images not considering the scenario (folder) label

#### After consultating with my doctor he has recommend me to go with the 
### Convolutional LSTM as it is convenient for time-series works
### And also can work multidimensionally which make it a good fit for image and video analysis

#### Define the model

# Structure Data Appropriately: Each input for the ConvLSTM needs to be shaped as (samples, time_steps, height, width, channels), where samples is the number of sequences, time_steps is the number of images in each sequence, height and width are the dimensions of each image, and channels refers to the number of channels (1 for grayscale, 3 for RGB).

In [22]:
X_train_s1, X_train_s2, y_train = prepare_data_for_model(train_grouped, target_shape=(522, 544))
X_test_s1, X_test_s2, y_test = prepare_data_for_model(test_grouped, target_shape=(522, 544))
X_val_s1, X_val_s2, y_val = prepare_data_for_model(validation_grouped, target_shape=(522, 544))

# Check the shape and ensure they match expected dimensions
print("Training shapes:", X_train_s1.shape, X_train_s2.shape, y_train.shape)
print("Testing shapes:", X_test_s1.shape, X_test_s2.shape, y_test.shape)
print("Validation shapes:", X_val_s1.shape, X_val_s2.shape, y_val.shape)

Training shapes: (16, 522, 544) (108, 522, 544) (124,)
Testing shapes: (6, 522, 544) (36, 522, 544) (42,)
Validation shapes: (16, 522, 544) (108, 522, 544) (124,)


In [23]:
### Here we need to introduce "padding" to work in the different lenghts of the 
### sequences as different sequences have a different number of images.
### paddding is the way of fixing this dimensionality problem

#Masking: After padding, using a Masking layer tells the model to ignore the padding during training, which helps in focusing on the meaningful data. Without masking, the model might learn the padding as a significant part of the input pattern, potentially skewing results.
### Padding at the beggining is the common practice when dealing with time series

In [24]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Input, Masking, ConvLSTM2D, BatchNormalization, Dense, Flatten, concatenate, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Lambda

# def pad_sequences_uniformly(X_s1, X_s2, max_len=None):
    
#     if not max_len: 
#         max_len = max(max(seq.shape[0] for seq in X_s1), max(seq.shape[0] for seq in X_s2))
#     X_s1_padded = [pad_sequences([seq], maxlen=max_len, dtype='float32', padding='post', truncating='post', value=0.0)[0] for seq in X_s1]
#     X_s2_padded = [pad_sequences([seq], maxlen=max_len, dtype='float32', padding='post', truncating='post', value=0.0)[0] for seq in X_s2]
    
#     return np.array(X_s1_padded), np.array(X_s2_padded)

def pad_image_sequences(sequences, maxlen=None, dtype='float32'):
    # Pad each sequence to the same length
    padded_sequences = pad_sequences(sequences, maxlen=maxlen, dtype=dtype, padding='post', truncating='post', value=0.0)
    return padded_sequences

# Determine the maximum sequence length across both S1 and S2
max_len = max([len(seq) for seq in X_train_s1] + [len(seq) for seq in X_train_s2])

# Pad S1 and S2 training data
X_train_s1_padded = pad_image_sequences(X_train_s1, maxlen=max_len)
X_train_s2_padded = pad_image_sequences(X_train_s2, maxlen=max_len)

# Ensure dimensions are correct
print("Shape of X_train_s1_padded:", X_train_s1_padded.shape)
print("Shape of X_train_s2_padded:", X_train_s2_padded.shape)
print("Shape of y_train:", y_train.shape)

Shape of X_train_s1_padded: (16, 522, 544)
Shape of X_train_s2_padded: (108, 522, 544)
Shape of y_train: (124,)


In [31]:
### Leverage S2 lenght train set 

def average_s2_sequences(s2_data, target_length):
    # s2_data is expected to be a list of arrays, each of which is a sequence of images
    averaged_sequences = []
    for sequence in s2_data:
        sequence_length = sequence.shape[0]
        if sequence_length > target_length:
            new_sequence = np.zeros((target_length, *sequence.shape[1:]))
            step = sequence_length // target_length
            for i in range(target_length):
                start_index = i * step
                end_index = start_index + step
                # Averaging frames to reduce the sequence length
                new_sequence[i] = np.mean(sequence[start_index:end_index], axis=0)
            averaged_sequences.append(new_sequence)
        else:
            averaged_sequences.append(sequence)  # No change if already matching or fewer frames
    return np.array(averaged_sequences)

In [32]:
# Example usage before training or testing:
# Assume X_train_s2 and X_train_s1 are your original datasets loaded and ready
target_length = X_train_s1.shape[1]  # Target length is the number of frames in S1 sequences
X_train_s2_adjusted = average_s2_sequences(X_train_s2, target_length)

In [33]:
input_shape = (None, 522, 544, 1)

In [34]:
### Working model



# Now define the input layers with the correct input shape
s1_input = Input(shape=input_shape, name='S1_input')
s2_input = Input(shape=input_shape, name='S2_input')

# Define the rest of your model architecture as previously
s1_branch = ConvLSTM2D(8, (3, 3), activation='relu', return_sequences=True)(s1_input)
s1_branch = BatchNormalization()(s1_branch)
s1_branch = ConvLSTM2D(4, (3, 3), activation='relu', return_sequences=False)(s1_branch)
s1_branch = Flatten()(s1_branch)

s2_branch = ConvLSTM2D(8, (3, 3), activation='relu', return_sequences=True)(s2_input)
s2_branch = BatchNormalization()(s2_branch)
s2_branch = ConvLSTM2D(8, (3, 3), activation='relu', return_sequences=False)(s2_branch)
s2_branch = Flatten()(s2_branch)

# Combine the outputs from both branches
combined = concatenate([s1_branch, s2_branch])

final_layer = Dense(8, activation='relu')(combined)
output = Dense(1, activation='sigmoid')(final_layer)

model = Model(inputs=[s1_input, s2_input], outputs=output)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 S1_input (InputLayer)          [(None, None, 522,   0           []                               
                                544, 1)]                                                          
                                                                                                  
 S2_input (InputLayer)          [(None, None, 522,   0           []                               
                                544, 1)]                                                          
                                                                                                  
 conv_lstm2d_8 (ConvLSTM2D)     (None, None, 520, 5  2624        ['S1_input[0][0]']               
                                42, 8)                                                        

In [35]:
def add_required_dimensions(X):
    # Add a time dimension if not present, and ensure there's a channel dimension
    # This is assuming X is a list of 3D arrays [samples, rows, cols]
    # We'll treat the entire set as a single batch with multiple time steps here for demonstration
    if X.ndim == 3:  # [samples, rows, cols]
        X = X[:, np.newaxis, :, :, np.newaxis]  # Reshape to [samples, 1 (time), rows, cols, 1 (channel)]
    elif X.ndim == 4:  # [samples, time, rows, cols]
        X = X[:, :, :, :, np.newaxis]  # Add channel dimension
    return X

In [36]:
# Apply the function
X_train_s1_padded = add_required_dimensions(X_train_s1_padded)
X_train_s2_padded = add_required_dimensions(X_train_s2_padded)

print("Adjusted Shape of X_train_s1_padded:", X_train_s1_padded.shape)
print("Adjusted Shape of X_train_s2_padded:", X_train_s2_padded.shape)

Adjusted Shape of X_train_s1_padded: (16, 1, 522, 544, 1)
Adjusted Shape of X_train_s2_padded: (108, 1, 522, 544, 1)


In [37]:
### Fit the model with the training sequences

In [38]:
model.fit([X_train_s1_padded, X_train_s2_padded], y_train, epochs=5, batch_size=1, validation_split=0.2)

Epoch 1/5
12/12 [==============================] - 12s 621ms/step - loss: 0.0551 - accuracy: 1.0000 - val_loss: 2.1169e-25 - val_accuracy: 1.0000
Epoch 2/5
12/12 [==============================] - 7s 571ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 1.0830e-17 - val_accuracy: 1.0000
Epoch 3/5
12/12 [==============================] - 7s 568ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 3.6747e-13 - val_accuracy: 1.0000
Epoch 4/5
12/12 [==============================] - 7s 564ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 5.7128e-13 - val_accuracy: 1.0000
Epoch 5/5
12/12 [==============================] - 7s 566ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 7.0521e-16 - val_accuracy: 1.0000


In [42]:
def adjust_s2_to_s1(X_s1, X_s2):
    num_s1 = X_s1.shape[0]
    num_s2 = X_s2.shape[0]
    
    #### Equal sizes
    adjusted_s2 = np.zeros((num_s1, *X_s2.shape[1:]))
    
    group_size = num_s2 // num_s1
    
    for i in range(num_s1):
        start_idx = i * group_size 
        end_idx = start_idx + group_size 
        
        adjusted_s2[i] = np.mean(X_s2[start_idx:end_idx], axis=0)
        
    return adjusted_s2

X_test_s2_adjusted = adjust_s2_to_s1(X_test_s1, X_test_s2)

In [ ]:
## Explanation

#     Averaging S2 images: This reduces the number of S2 images to match the count of S1 images. Each new S2 image in adjusted_s2 is an average of several consecutive original S2 images.
#     Consistent input sizes: Both S1 and S2 inputs to the model now have the same number of samples, aligning with the model's expectation and avoiding the cardinality error.

# This approach should effectively handle the disparity in the number of S1 and S2 images, allowing your model to make predictions based on both inputs synchronized by their adjusted counts.

In [43]:
### Predictions 

In [45]:
# Assuming X_test_s1 and X_test_s2 are your test sets for the two types of inputs and are properly formatted
predicted_probabilities = model.predict([X_test_s1, X_test_s2_adjusted])
predicted_labels = (predicted_probabilities > 0.5).astype(int)

1/1 [==============================] - 1s 1s/step


In [47]:
print(predicted_labels)

[[0]
 [0]
 [0]
 [0]
 [0]
 [0]]
